In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature= 0.1,
    streaming= True,
    callbacks= [
        StreamingStdOutCallbackHandler()
    ]
)

# t = PromptTemplate.from_template("What is the capital of {country}")

'''아래 코드는 의미상 위에 작성한 코드와 같다. 편의상 위의 코드 작성법 사용'''

# t = PromptTemplate(
#     template = "What is the capital of {country}",
#     input_variables= ["country"]
# )

'아래 코드는 의미상 위에 작성한 코드와 같다. 편의상 위의 코드 작성법 사용'

In [8]:
'''FewShotPromptTemplate - 예제를 LLM 에게 제공함으로써 답변 품질을 높인다.
예를들어 고객센터 챗복 구축 시 데이터베이스 등에 대화 기록 등 사용 가능한 데이터가 있는 경우 prompt를 작성하는 것보다 효과적일 수 있다.'''
from langchain.prompts.few_shot import FewShotPromptTemplate 


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
        {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
            {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athenes
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt, # 어떻게 전달한 예제들을 형식화 할지 알려줌
    examples= examples,
    suffix= "Human: What do you know about {country}?", # example 맨 마지막에 오는 질문
    input_variables=["country"], # 템플릿에서 사용할 입력 변수. 유효성 검사(validation)에서 사용
)
# prompt.format(country="Korea")

chain = prompt | chat
chain.invoke({"country": "Korea"})



AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

In [23]:
'''FewShotChatMessagePromptTemplate'''

from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
        {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
            {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athenes
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
    
])

example_prompt = FewShotChatMessagePromptTemplate(
    examples= examples,
    example_prompt= example_prompt, # 어떻게 전달한 예제들을 형식화 할지 알려줌
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Your are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({"country": "Korea"})

I know this:
        South Korea:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

        North Korea:
        Capital: Pyongyang
        Language: Korean
        Food: Kimchi and Rice
        Currency: North Korean Won

AIMessageChunk(content='I know this:\n        South Korea:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won\n\n        North Korea:\n        Capital: Pyongyang\n        Language: Korean\n        Food: Kimchi and Rice\n        Currency: North Korean Won')

In [14]:
'''example selector 사용 예제'''

from typing import Any, Dict, List
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """
    },
        {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
            {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athenes
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    },
]
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        return self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        
        return [choice(self.examples)]
    

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples= examples, 
    # example_prompt= example_prompt,
    # max_length= 180,
)

# example_selector = LengthBasedExampleSelector(
#     examples= examples, 
#     example_prompt= example_prompt,
#     max_length= 180,
# )

prompt = FewShotPromptTemplate(
    example_prompt= example_prompt, # 어떻게 전달한 예제들을 형식화 할지 알려줌
    example_selector= example_selector,
    suffix= "Human: What do you know about {country}?", # example 맨 마지막에 오는 질문
    input_variables=["country"], # 템플릿에서 사용할 입력 변수. 유효성 검사(validation)에서 사용
)
prompt.format(country="Korea")



'Human: What do you know about Greece?\nAI: \n        I know this:\n        Capital: Athenes\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Korea?'

In [5]:
# Prompt를 다른 파일로 만들어두고 불러오는 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature= 0.1,
    streaming= True,
    callbacks= [
        StreamingStdOutCallbackHandler()
    ]
)

prompt.format(country="Korea")

'What is the capital of Korea'